# Author John Payne
# Step 1
* Generate a response using the Casseopeia wrapper

In [142]:
import random
import cassiopeia as cass
import requests as re

def getAPI_key():
    f = open("../api_key.txt", "r")
    return f.read()


In [166]:
url = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=RGAPI-bb3ceae6-9bc6-42b0-99f3-021bcd278a84'
api_key = getAPI_key()
    
res = re.get(url)
res.status_code

200

In [177]:
def get_challenger_player_stats():
    url = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }

    
    return re.get(url, headers = header_dict).json()

In [216]:
challenger_ladder = get_challenger_player_stats()

In [196]:
def get_challenger_player_id(player_id):
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + player_id
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    

        
    return re.get(url, headers = header_dict).json()

In [214]:
player_info = get_challenger_player_id('K_vwP8v9V5HxuPw2bPNmiXATqtiTGD8eh2ofz1QqHtKtBco')
player_info

{'id': 'K_vwP8v9V5HxuPw2bPNmiXATqtiTGD8eh2ofz1QqHtKtBco',
 'accountId': 'pfxVaoTIy5DE0O2ckuqgI5Psc9eDHZAE4wyTGf_sOhHVgzY',
 'puuid': 'KO0YCUqpN8cOSIyAkP-PGK9XFeTonEb4pkrxAQta62nCtJM1qZeabD_2mRMXcSaOsPwKuWo2IN27UQ',
 'name': 'AHº',
 'profileIconId': 3798,
 'revisionDate': 1661924504000,
 'summonerLevel': 533}

In [281]:
def get_matches(puuid):
    url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    
    params_dict = {
        #Only picking games played on patch 12.15, 08-10-2022 to 0-24-2022
        'startTime' : 1660125600,
        'endTime' : 1661335200,
        
        #Only picking ranked 5v5 games
        'queue' : 420,
        'type' : 'ranked',
        
        #Only grabbing the first 50 results
        'start' : 0,
        'count' : 50
    }
    return re.get(url, headers = header_dict, params = params_dict).json()

In [282]:
game_list = get_matches('KO0YCUqpN8cOSIyAkP-PGK9XFeTonEb4pkrxAQta62nCtJM1qZeabD_2mRMXcSaOsPwKuWo2IN27UQ')

In [220]:
def get_match_info(match):
    url = 'https://americas.api.riotgames.com/lol/match/v5/matches/' + match
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    
    return re.get(url, headers = header_dict).json()
    

In [221]:
get_match_info('NA1_4422823254')

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4422823254',
  'participants': ['q-pD7H6HS43Zjd-JWbVa6QOph6Cwm38Wi64M_doKyy9o0X7B9y5vl_mbTTop_M1vDqHjOKdLb25cGg',
   'MbjhzeooY_bgNaaiUUhYFM91_4AF_3Ha9kGgjKUbdTuQmP9r339U3wNlxpCDdZC1ntFS-OX2tsDb7Q',
   'VkZbVjgzI3KV8jO6SchZN-HvwX1gAElJNyVJd1TVGbXlSG-whHAs2xaB9AdijuFXdFJ6nr5BpYA5XA',
   'KO0YCUqpN8cOSIyAkP-PGK9XFeTonEb4pkrxAQta62nCtJM1qZeabD_2mRMXcSaOsPwKuWo2IN27UQ',
   'W-xz51knhJiDl0e_aFFBOXRZ99-O6UxbLwacFpaD7wn2cpqc_qe-fb1CeF9J1mtU_H4k90orwNA5zQ',
   'BuSvd8XY1f1QBw2Y_km6x_gd-J_WCE6BY2Ov0d5TdI1X8S98D91IrPujQs-NJhf1Ou0rfsmmek1wTw',
   'A4kDo_sLQ69v8Jg-LQ0HWIs3VSTGp44XyOYH4YvJW6_AuhWUpucgh3XnGyfC7IGzxL5GJeq4yTyAtg',
   'bPnBqIppCsdFThHLVP6KYTurgiMZQl3xgOS56odWWawo9FcchJ8yx1x9I6Z0v0YvFcvBWlnAuPELjQ',
   'MFoZ-kJ5jsgRu_m7O0XlSXLXuLh7BJf4d4odWr2jh2a0OlD1p_uqB9ZdG951CYQtrequF8IKVrnMkQ',
   'veqKmOQH2eDzmF62mveeVIEvaOXA50K35lGIAgubElz_y37LVGpkHbeF5Lo1BPbb1E6FeZaMEghzcw']},
 'info': {'gameCreation': 1662013562917,
  'gameDuration': 1814,
  

In [284]:
class Summoner:
    
    def __init__(self, summonerId):
        self.summonerId = summonerId
        
        for value in challenger_ladder['entries']:
            if value['summonerId'] == self.summonerId:
                self.leaguepoints = value['leaguePoints']
                self.wins = value['wins']
                self.losses = value['losses']
                self.hotStreak = value['hotStreak']
                self.summonerName = value['summonerName']
        
        #Returns a dictionary
        self.accounts = get_challenger_player_id(summonerId)
        self.puuid = self.accounts['puuid']
        self.summonerLevel = self.accounts['summonerLevel']
        
        #Returns a list
        self.matches = get_matches(self.puuid)
        
    

In [285]:
sum1 = Summoner('n1YGh4aePr5T0VHUHFWnfuqgGi1BkswPBKGxII-eQIQnQH0')
sum2 = Summoner('K_vwP8v9V5HxuPw2bPNmiXATqtiTGD8eh2ofz1QqHtKtBco')

In [249]:
sum1_match = sum1.matches
sum2_match = sum2.matches

In [270]:
summoner_classes = []
value = 1
while value <= 300:
    value = str(value)
    summoner_classes.append('sum' + value)
    value = int(value)
    value += 1

In [253]:
total_matches = sum1_match + sum2_match
total_matches = list(set(total_matches))

40
40


In [287]:
sum1.matches

['NA1_4407105995',
 'NA1_4405293982',
 'NA1_4405232301',
 'NA1_4403614443',
 'NA1_4402680445',
 'NA1_4402434827',
 'NA1_4402421516',
 'NA1_4402349412',
 'NA1_4402345382',
 'NA1_4401369255']

In [258]:
challenger_ladder['entries'][0]['summonerId']

'n1YGh4aePr5T0VHUHFWnfuqgGi1BkswPBKGxII-eQIQnQH0'